In [20]:
import torch
import torch.optim as optim
import numpy as np

n = 5  # Number of AMMs
k = 3  # Number of tokens

quantities = [[10, 20, 10], [60, 40, 100], [100, 150, 190], [5, 1, 2], [1000, 1500, 2000]]  # Example initial quantities for each AMM
initial_quantities = torch.tensor(quantities, requires_grad=False)

# Define Delta for the first n-1 AMMs only, since the last one will be derived
Delta = torch.zeros((n-1, k), requires_grad=True)

def calc_full_delta(Delta):
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    last_row_adjustment = -torch.sum(Delta, axis=0, keepdims=True)
    # Combine adjustments to get the full Delta matrix
    return torch.cat((Delta, last_row_adjustment), axis=0)

# Objective function considering the new Delta definition
def utilities(Delta):
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    full_delta = calc_full_delta(Delta)
    new_quantities = initial_quantities + full_delta
    products = new_quantities.prod(dim=1)  # Product of token quantities in each AMM
    return products

initial_utility = utilities(Delta).detach()

print(f"Initial utility: {initial_utility}")


def utility_pct_changes(Delta):
    utes = utilities(Delta)
    pct_change = (utes - initial_utility) / initial_utility
    return pct_change

def utility(Delta):
    return utility_pct_changes(Delta).sum()


def difference_between_utility_changes(Delta):
    # penalize differences between utility changes (ideally they should be equal)
    utes = utility_pct_changes(Delta)
    diffs = utes - utes.mean()
    return diffs.abs().sum()

# Optimization setup
optimizer = optim.Adam([Delta], lr=0.05)

# Optimization loop
for step in range(20000):
    optimizer.zero_grad()
    
    # Calculate utility
    loss = -10*utility(Delta)  # Negative sign for maximization

    # penalty for nonnegative values of new quantities
    relu_loss = torch.sum(torch.relu(-initial_quantities - calc_full_delta(Delta)))
    loss += 10000*relu_loss

    # penalty for differences between utility changes (ideally they should be equal)
    # difference_loss = difference_between_utility_changes(Delta)
    difference_loss = utility_pct_changes(Delta).var()
    loss += 10000 * difference_loss
    
    # Backward pass and update
    loss.backward()
    optimizer.step()
    
    if step % 1000 == 0:
        print(f"Step {step}, Utility: {-loss.item()}, Relu Loss: {relu_loss}, Difference Loss: {difference_loss}")

# After optimization, calculate the full Delta including the last row
full_delta = calc_full_delta(Delta)

print("Optimized adjustments (Delta):")
print(full_delta.data)
print("Original token quantities:")
print(initial_quantities)
print("New token quantities:")
data = (initial_quantities + full_delta).data
print(data)
data_np = data.numpy()

price_ratio1 = data_np[0, 0] / data_np[0, 1]
price_ratio2 = data_np[1, 0] / data_np[1, 1]


price_ratios = data_np[:, 0] / data_np[:, 1]

print("price ratios:")
print(price_ratios)

print("new utilities:")
print(utilities(Delta).data)

print("utility changes:")
print(utility_pct_changes(Delta).data)

Initial utility: tensor([2.0000e+03, 2.4000e+05, 2.8500e+06, 1.0000e+01, 3.0000e+09])
Step 0, Utility: -0.0, Relu Loss: 0.0, Difference Loss: 0.0
Step 1000, Utility: 0.29858988523483276, Relu Loss: 0.0, Difference Loss: 1.404762952006422e-05
Step 2000, Utility: 0.4773241877555847, Relu Loss: 0.0, Difference Loss: 1.403839087288361e-05
Step 3000, Utility: 0.5095955729484558, Relu Loss: 0.0, Difference Loss: 1.5832585631869733e-05
Step 4000, Utility: 0.6049559116363525, Relu Loss: 0.0, Difference Loss: 1.4150849892757833e-05
Step 5000, Utility: 0.6222225427627563, Relu Loss: 0.0, Difference Loss: 1.4084327631280757e-05
Step 6000, Utility: 0.6215496063232422, Relu Loss: 0.0, Difference Loss: 1.665454874455463e-05
Step 7000, Utility: 0.6352892518043518, Relu Loss: 0.0, Difference Loss: 1.3767748896498233e-05
Step 8000, Utility: 0.6383957862854004, Relu Loss: 0.0, Difference Loss: 1.4310629921965301e-05
Step 9000, Utility: 0.6401913166046143, Relu Loss: 0.0, Difference Loss: 1.4059258319321

In [61]:
import torch
import torch.optim as optim
import numpy as np

n = 3  # Number of AMMs
k = 3  # Number of tokens

# quantities = [[10, 20, 10], [60, 40, 100], [100, 150, 190], [5, 1, 2], [1000, 1500, 2000]]  # Example initial quantities for each AMM
quantities = [[10, 20, 10], [60, 40, 100], [10, 10, 10]]  # Example initial quantities for each AMM

initial_quantities = torch.tensor(quantities, requires_grad=False)

# Define Delta for the first n-1 AMMs only, since the last one will be derived
Delta = torch.zeros((n-1, k), requires_grad=True)

def calc_full_delta(Delta):
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    last_row_adjustment = -torch.sum(Delta, axis=0, keepdims=True)
    # Combine adjustments to get the full Delta matrix
    return torch.cat((Delta, last_row_adjustment), axis=0)

# Objective function considering the new Delta definition
def utilities(Delta):
    # Calculate adjustments for the last AMM as the negation of the sum of the others
    full_delta = calc_full_delta(Delta)
    new_quantities = initial_quantities + full_delta
    products = new_quantities.prod(dim=1)  # Product of token quantities in each AMM
    return products

initial_utility = utilities(Delta).detach()

print(f"Initial utility: {initial_utility}")


def utility_pct_changes(Delta):
    utes = utilities(Delta)
    pct_change = (utes - initial_utility) / initial_utility
    return pct_change

def utility(Delta):
    return utility_pct_changes(Delta).sum()

def utility2(Delta):
    new_utilities = utilities(Delta)
    utility_change = new_utilities - initial_utility
    # first n-1 rows: relu penalty for negative values. sum of the relus
    relu_penalty = torch.relu(-utility_change[:-1]).sum()
    # last row: maximize utility change
    utility_change_penalty = -utility_change[-1]
    return relu_penalty, utility_change_penalty



    

def difference_between_utility_changes(Delta):
    # penalize differences between utility changes (ideally they should be equal)
    utes = utility_pct_changes(Delta)
    diffs = utes - utes.mean()
    return diffs.abs().sum()

# Optimization setup
optimizer = optim.Adam([Delta], lr=0.03)

# Optimization loop
for step in range(20000):
    optimizer.zero_grad()
    
    # Calculate utility
    # loss = -10*utility(Delta)  # Negative sign for maximization
    loss = 0
    # # penalty for nonnegative values of new quantities
    relu_loss1 = torch.sum(torch.relu(-initial_quantities - calc_full_delta(Delta)))
    loss += 10000000*relu_loss1

    # # penalty for differences between utility changes (ideally they should be equal)
    # # difference_loss = difference_between_utility_changes(Delta)
    # difference_loss = utility_pct_changes(Delta).var()
    # loss += 10000 * difference_loss
    relu_loss2, utility_change_penalty = utility2(Delta)
    loss += 1000*utility_change_penalty

    loss += 10000*relu_loss2
    
    # Backward pass and update
    loss.backward()
    optimizer.step()
    
    if step % 1000 == 0:
        print(f"Step {step}, Utility: {-loss.item()}, negative loss: {relu_loss1}, Relu Loss: {relu_loss2}, Difference Loss: {utility_change_penalty}")

# After optimization, calculate the full Delta including the last row
full_delta = calc_full_delta(Delta)

print("Optimized adjustments (Delta):")
print(full_delta.data)
print("Original token quantities:")
print(initial_quantities)
print("New token quantities:")
data = (initial_quantities + full_delta).data
print(data)
data_np = data.numpy()

price_ratio1 = data_np[0, 0] / data_np[0, 1]
price_ratio2 = data_np[1, 0] / data_np[1, 1]


price_ratios = data_np[:, 0] / data_np[:, 1]

print("price ratios:")
print(price_ratios)

print("original utilities:")
print(initial_utility)

print("new utilities:")
print(utilities(Delta).data)

print("utility changes:")
print(utility_pct_changes(Delta).data)

Initial utility: tensor([  2000., 240000.,   1000.])
Step 0, Utility: -0.0, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -0.0
Step 1000, Utility: 32718.3828125, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -32.7183837890625
Step 2000, Utility: 163021.125, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -163.0211181640625
Step 3000, Utility: 215167.71875, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -215.167724609375
Step 4000, Utility: 344442.875, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -344.44287109375
Step 5000, Utility: 504137.9375, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -504.137939453125
Step 6000, Utility: 488778.3125, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -488.7783203125
Step 7000, Utility: 678534.9375, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -678.534912109375
Step 8000, Utility: 680282.5, negative loss: 0.0, Relu Loss: 0.0, Difference Loss: -680.282470703125
Step 9000, Utility: 693789.4375, negat

In [44]:
import cvxpy as cp
import numpy as np

# Example matrix A (n x k)
A = np.array([[0.25, 0.75, 0.5],
              [0.5, 0.25, 0.75]])

n, k = A.shape

# Variable matrix B (n x k), the matrix you're solving for
B = cp.Variable((n-1, k))

# last row is derived from the others such that each column sums to zero
B = cp.vstack([B, -cp.sum(B, axis=0, keepdims=True)])


# Auxiliary variable representing the common dot product value
common_dot_product = cp.Variable()

# Constraints
# 1. Columns of B sum to zero
# columns_sum_to_zero = [cp.sum(B, axis=0) == 0]
# rows are unit vectors
rows_are_unit_vectors = [cp.norm(B[i,:], 2) == 1 for i in range(n)]

# 2. All row dot products are equal to the common dot product value
row_dot_products_equal = [cp.sum(cp.multiply(A[i,:], B[i,:])) == common_dot_product for i in range(n)]

# Combine all constraints
constraints = row_dot_products_equal + rows_are_unit_vectors

# Objective: Maximize the common dot product
objective = cp.Maximize(common_dot_product)

# Define the problem and solve it
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB, verbose=True)

print("Optimal B:", B.value)
print("Common dot product value:", common_dot_product.value)


                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Apr 04 07:53:54 PM: Your problem has 4 variables, 4 constraints, and 0 parameters.
(CVXPY) Apr 04 07:53:54 PM: It is compliant with the following grammars: 
(CVXPY) Apr 04 07:53:54 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 04 07:53:54 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 04 07:53:54 PM: Your problem is compiled with the CPP canonicalization backend.


DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
Pnorm(Vstack(var381, -Sum(var381, 0, True))[0, 0:3], 2) == 1.0 , because the following subexpressions are not:
|--  Pnorm(Vstack(var381, -Sum(var381, 0, True))[0, 0:3], 2) == 1.0
Pnorm(Vstack(var381, -Sum(var381, 0, True))[1, 0:3], 2) == 1.0 , because the following subexpressions are not:
|--  Pnorm(Vstack(var381, -Sum(var381, 0, True))[1, 0:3], 2) == 1.0

In [43]:
import numpy as np
from scipy.optimize import minimize

# Example matrix A
A = np.array([[0.25, 0.75, 0.5], [0.5, 0.25, 0.75], [0.6, 0.4, 0]])

n, k = A.shape

# Define the objective function focusing only on optimizing the first n-1 rows of B
def objective(B_flat):
    B = B_flat.reshape((n-1), k)
    # Derive the last row to ensure column sums of B are zero
    last_row = -np.sum(B, axis=0)
    B_complete = np.vstack([B, last_row])
    # Calculate dot products of A and B
    dot_products = np.sum(A * B_complete, axis=1)
    # Objective: maximize the minimum dot product or any other suitable objective
    return -np.min(dot_products)

# No constraints on the row sums of B in this setup



# Initial guess for B (only for the first n-1 rows)
B_initial = np.random.rand((n-1) * k)

# Solve the optimization problem
result = minimize(objective, B_initial, method='SLSQP')

if result.success:
    # Reshape the result to match the first n-1 rows of B
    B_optimized = result.x.reshape((n-1), k)
    # Calculate the last row based on the optimized first n-1 rows
    last_row = -np.sum(B_optimized, axis=0)
    # Combine to get the complete B
    B_complete = np.vstack([B_optimized, last_row])
    print("Optimal B:", B_complete)
    # Optionally, calculate the uniform dot products as an additional check
    dot_products = np.sum(A * B_complete, axis=1)
    print("Dot products:", dot_products)
else:
    print("Optimization failed:", result.message)


Optimal B: [[-1.73274486e+08  1.08344010e+08  1.91880289e+08]
 [-8.84315652e+07 -8.90848887e+07  2.71174599e+08]
 [ 2.61706051e+08 -1.92591216e+07 -4.63054888e+08]]
Dot products: [1.33879531e+08 1.36893945e+08 1.49319982e+08]


In [34]:
import numpy as np
from scipy.optimize import minimize

# Example matrix A
A = np.array([[0.25, 0.75, 0.5], [0.5, 0.25, 0.75]])

n, k = A.shape

# Objective function: Negative sum of dot products to maximize the sum
def objective(B_flat):
    B = B_flat.reshape(n, k)
    dot_products = np.sum(A * B, axis=1)
    # We try to maximize the minimum dot product
    return -np.min(dot_products)

# Constraints
# Columns of B sum to zero
def constraint_columns_sum_to_zero(B_flat):
    B = B_flat.reshape(n, k)
    return np.sum(B, axis=0)

# Initial guess
B_initial = np.random.rand(n*k)

# Constraints dictionary
con_columns = {'type': 'eq', 'fun': constraint_columns_sum_to_zero}

# Optimize
result = minimize(objective, B_initial, constraints=[con_columns], method='SLSQP')

B_optimal = result.x.reshape(n, k)

print("Optimal B:", B_optimal)
print("Objective value:", -result.fun)  # Remember we minimized the negative sum

(B_optimal * A).sum(axis=1)


Optimal B: [[-1.22166149e+14  2.57922534e+14 -1.19448102e+14]
 [ 1.22166149e+14 -2.57922534e+14  1.19448102e+14]]
Objective value: 86188518002819.38


array([1.03176312e+14, 8.61885180e+13])